<a href="https://colab.research.google.com/github/shanxS/MLPlayground/blob/main/gpt_dev_0_01_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [21]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyper parameters
batch_size = 32
block_size = 8
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
#----------------

torch.manual_seed(1337)

#!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(''.join(text))))
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for s,i in stoi.items()}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])
vocab_size = len(chars)

data = torch.tensor(encode(text))
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data)-block_size, (batch_size,))
  x = torch.stack( [data[i: i+block_size] for i in ix] )
  y = torch.stack( [data[i+1:i+block_size+1] for i in ix] )
  return x, y

@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for K in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[K] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out




# Bigram model

In [22]:
class BigramLangugageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    '''
    1st argument is size of dict
    2nd argument is size of embedding for each input token
    So the dim is vocab_size x C, where C is Channel
    '''
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, input, target = None):
    '''
    This is the output. Literally just dot product of input (B x T) and embedding table (vocab_size x channel) to get embedding
    I think it internally converts input into one hot encoding of size (B x T x vocab_size)
    B: number of inputs
    T: size of each input
    C: channel
    '''
    # print(input)
    # print(target)
    
    logits = self.token_embedding_table(input) # (B x T x C)

    if target is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      target = target.view(B*T)
      loss = F.cross_entropy(logits, target)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, _ = self(idx)
      logits = logits[:, -1, :] # I dont understand why last index is predicted token
      logits = logits.exp()
      prob = logits / logits.sum(1, keepdims = True)
      idx_next = torch.multinomial(prob, num_samples=1, replacement=True)
      idx = torch.cat((idx, idx_next), dim =1)
    return idx

# Model training & eval

In [23]:
model = BigramLangugageModel(vocab_size)
m = model.to(device)

optimizer = torch.optim.AdamW(m.parameters(), learning_rate)

In [27]:
for iter in range(max_iters):
  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f"step:{iter} training loss:{losses['train']} val loss:{losses['val']}")

  xb, yb = get_batch('train')    
  logits, loss = m(xb, yb)
  loss.backward()
  optimizer.step()

step:0 training loss:3.056396484375 val loss:3.1936521530151367
step:300 training loss:3.04364013671875 val loss:3.1544766426086426
step:600 training loss:3.0537619590759277 val loss:3.1870498657226562
step:900 training loss:3.0858631134033203 val loss:3.2429065704345703
step:1200 training loss:3.193424701690674 val loss:3.369863271713257
step:1500 training loss:3.165262460708618 val loss:3.229585886001587
step:1800 training loss:3.177609920501709 val loss:3.321354866027832
step:2100 training loss:3.278136968612671 val loss:3.4377191066741943
step:2400 training loss:3.2006475925445557 val loss:3.4026148319244385
step:2700 training loss:3.3570079803466797 val loss:3.5171005725860596


# Generate

In [26]:
context = torch.zeros((1,1), dtype = torch.long, device = device)
print(decode(m.generate(context, max_new_tokens = 400)[0].tolist()))


Corver nereve goanoravit, cinkerin t
Mus n weve ineverve t userar t s he!
CK:
Ale
Dine a ep m:
Doratarin.
DWe ir imatle ts
Awors ce g t n it, gdor d, o he hileas t th wiever no ore thorcackep th
Foro; therimirend, is thinanimallllir evier o t nthoraviconorrithoverer hanerchous!
Coo iat s k,
MERERWeruse n e ndlouerd.
An orits thir tthorp ck nyorde t orareprind, ichor whove ne eve t or serind tho t 
